# LVV-T297: Absolute Astrometric Performance

**Written By: Bryce Kalmbach**

**Last updated: 08-28-2019**

**Tested on Stack Version: w_2019_33**

## Requirements:

[OSS-REQ-0388](https://docushare.lsst.org/docushare/dsweb/Get/LSE-030#page=66)

Median error in absolute position for each axis, RA and DEC, shall be less than 50 milliarcseconds.

## Proposed Test Case:

1. Take images from region overlapping the Gaia footprint.  Repeat at multiple airmasses.

2. Perform source detection and astrometric measurement on images from step 1

3. Cross-match catalog from step 2 with Gaia catalog.  Select sources that are consistent with zero proper motion (according to Gaia).

4. Verify that the median error of the LSST positions (relative to the Gaia positions) is **50 milliarcseconds in RA, Dec independently**

### Import necessary tools

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from lsst.daf.persistence import Butler
import lsst.daf.persistence as daf_persistence

from astropy.coordinates import SkyCoord, Angle
from astropy import units as u

from lsst.meas.algorithms import (LoadIndexedReferenceObjectsTask, 
                                  LoadIndexedReferenceObjectsConfig)

import lsst.geom as geom
from lsst.meas.astrom import DirectMatchTask

In [ ]:
# Make our plots nice and readable
plt.rcParams.update({'font.size': 18})

### Set parameters for testing

* `test_bandpass`: The notebook will set up to test astrometry in this bandpass

In [ ]:
test_bandpass = 'HSC-R'

### Identify HSC Data to use

We want to get data from a single visit for this requirement so we choose a visit from the HSC Wide dataset. https://hsc-release.mtk.nao.ac.jp/doc/index.php/database/ has info 
on which tracts are included in the Wide data. We randomly choose tract 9348 for testing. To choose a different band for testing change `band` below.

In [ ]:
# Load a butler for the HSC Wide data
depth = 'WIDE'
band = test_bandpass
butler = daf_persistence.Butler('/datasets/hsc/repo/rerun/DM-13666/%s/'%(depth))

In [ ]:
# Find a visit in the WIDE data for specified band in the tract 9348
warp_list = os.listdir('/datasets/hsc/repo/rerun/DM-13666/WIDE/deepCoadd/%s/9348/0,0' % band)
warp_list.sort()
visit = int(warp_list[0].split('-')[-1].split('.')[0])

In [ ]:
subset = butler.subset('src', filter=band, visit=visit)

In [ ]:
# Load in sources from visit making exceptions for bad ccd 9 and focusing ccds.
hsc_sources_df = None
ccd_lims = []
for dataId in subset.cache:
    if dataId['ccd'] % 10 == 0:
        print('On CCD #%i' % dataId['ccd'])
    try:
        src_cat = butler.get('src', dataId=dataId)
        src_cat_df = src_cat.asAstropy().to_pandas()
        src_cat_df['ccd'] = dataId['ccd']
        if hsc_sources_df is None:
            hsc_sources_df = pd.DataFrame([], columns=src_cat_df.columns)
            hsc_sources_df = hsc_sources_df.append(src_cat_df, sort=False)
        else:
            hsc_sources_df = hsc_sources_df.append(src_cat_df, sort=False)
        ccd_lims.append([np.degrees(np.min(src_cat_df['coord_ra'])),
                         np.degrees(np.max(src_cat_df['coord_ra'])),
                         np.degrees(np.min(src_cat_df['coord_dec'])),
                         np.degrees(np.max(src_cat_df['coord_dec']))])
    except daf_persistence.butlerExceptions.NoResults as inst:
        print('No results for CCD #%i' % dataId['ccd'])

In [ ]:
# Total number of HSC Sources
len(hsc_sources_df)

In [ ]:
# Set up a bounding box for Gaia data retrieval using the edges of the CCDs available
ccd_lims = np.array(ccd_lims)
ra_min, ra_max, dec_min, dec_max = np.min(ccd_lims[:,0])-0.5, np.max(ccd_lims[:,1])+0.5, np.min(ccd_lims[:,2])-0.5, np.max(ccd_lims[:, 3])+0.5

In [ ]:
hsc_sources_coords = SkyCoord(hsc_sources_df['coord_ra']*u.rad, hsc_sources_df['coord_dec']*u.rad)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.scatter(hsc_sources_coords.ra.deg, hsc_sources_coords.dec.deg, s=8, lw=0)
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
plt.title('HSC Sources in Visit %i' %visit)
ax = plt.gca()
ax.set_xticks(ax.get_xticks()[1::2]) # Clean up ticks in RA

### Load Gaia data

We load the Gaia reference catalog to match against.

In [ ]:
config = LoadIndexedReferenceObjectsConfig()

In [ ]:
config.ref_dataset_name='gaia_DR2'

In [ ]:
config.filterMap = {}
for source in ('u', 'g', 'r', 'i', 'z', 'y'):
    config.filterMap[source] = "phot_g_mean"

In [ ]:
ref_task = LoadIndexedReferenceObjectsTask(butler, config=config)

In [ ]:
ra_mid = (ra_min + ra_max)/2.
dec_mid = (dec_min + dec_max)/2.

In [ ]:
ra_mid, dec_mid

In [ ]:
ref_cat = ref_task.loadSkyCircle(geom.SpherePoint(geom.Angle(ra_mid, geom.degrees), 
                                                  geom.Angle(dec_mid, geom.degrees)),
                                 geom.Angle(1., geom.degrees), filterName='g')

In [ ]:
ref_cat_df = ref_cat.refCat.asAstropy().to_pandas()

In [ ]:
ref_cat_df.head()

In [ ]:
gaia_coords = SkyCoord(ref_cat_df['coord_ra']*u.rad, ref_cat_df['coord_dec']*u.rad)

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.scatter(gaia_coords.ra.deg, gaia_coords.dec.deg, s=8, lw=0)
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
plt.title('Possible Gaia Sources in Visit %i' %visit)
ax = plt.gca()
ax.set_xticks(ax.get_xticks()[1::2]) # Clean up ticks in RA

### Use astropy to match

We will use the `match_to_catalog_sky` method from astropy to do the catalog match.

In [ ]:
def get_best_catalog_match(base_cat, ref_cat):
    
    """
    `match_to_catalog_sky` will give a match for every object in the base_cat. We only want to keep the best match to an object in the ref catalog.
    This function gives the ordered indices for the best match in base_cat to an object in the ref_cat and the separation measured.
    
    Inputs
    ------
    base_cat: astropy SkyCoord object
        The catalog with ra, dec locations of objects we observed and now want to match to a reference catalog.
        
    ref_cat: astropy SkyCoord object
        The ra, dec coordinates of objects from the reference catalog (the "truth" catalog).
        
    Returns
    -------
    base_cat_idx: list of integers
        The indices of the objects from base_cat that are the best ("closest") match to an object in the ref_cat.
        
    ref_cat_idx: list of integers
        The indices of objects in the ref_cat that matched to an observed object in the base_cat.
        
    single_match_seps: astropy Angle object
        The angular separation between the locations of the matched objects in the two catalogs.
    """
    
    idx, sep2d, sep3d = base_cat.match_to_catalog_sky(ref_cat)
    unique_idx = np.unique(idx)
    base_cat_idx = []
    ref_cat_idx = []
    single_match_seps = []
    for index in unique_idx:
        base_cat_rows = np.where(idx == index)[0]
        seps = sep2d[base_cat_rows]
        min_sep_idx = np.argmin(seps)
        base_cat_idx.append(base_cat_rows[min_sep_idx])
        ref_cat_idx.append(index)
        single_match_seps.append(seps[min_sep_idx])
        
    single_match_seps = Angle(single_match_seps)
        
    return base_cat_idx, ref_cat_idx, single_match_seps

In [ ]:
base_idx, ref_idx, matched_seps = get_best_catalog_match(hsc_sources_coords, gaia_coords)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(hsc_sources_coords[base_idx].ra.deg, hsc_sources_coords[base_idx].dec.deg, c=matched_seps.arcsec*1000, s=20, vmax=50, vmin=0)
cb = plt.colorbar()
plt.xlabel('RA (deg)')
plt.ylabel('dec (deg)')
cb.set_label('Distance to match (milliarcsec)')
ax = plt.gca()
ax.set_xticks(ax.get_xticks()[1::2]) # Clean up ticks in RA

### Plot results against requirements

In [ ]:
# Look at overall separation
fig = plt.figure(figsize=(10, 8))
n, bins, _ = plt.hist(matched_seps.arcsec, label='Gaia Objects', range=(0, 0.1), bins=20)
plt.axvline(np.median(matched_seps.arcsec), 0, np.max(n), c='r', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*np.median(matched_seps.arcsec)))
plt.title('Test of Absolute Astrometry')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

In [ ]:
# Requirement specifies looking at RA and dec inpendently
sep_ra = gaia_coords.ra.arcsec[ref_idx] - hsc_sources_coords.ra.arcsec[base_idx]
sep_dec = gaia_coords.dec.arcsec[ref_idx] - hsc_sources_coords.dec.arcsec[base_idx]
ra_median_sep = np.median(np.abs(sep_ra))
dec_median_sep = np.median(np.abs(sep_dec))

In [ ]:
# Requirement specifies RA, dec individually so we look at those here
fig = plt.figure(figsize=(20, 8))

fig.add_subplot(1,2,1)
n, bins, _ = plt.hist(np.abs(sep_ra), label='Gaia Objects', range=(0, 0.1), bins=20)
plt.axvline(ra_median_sep, 0, np.max(n), c='k', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*ra_median_sep))
plt.axvline(.05, 0, np.max(n), label='Requirement =  50 milliarcsec', c='r', lw=4)
plt.title('Test of Absolute Astrometry RA')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

fig.add_subplot(1,2,2)
n, bins, _ = plt.hist(np.abs(sep_dec), label='Gaia Objects', range=(0, 0.1), bins=20)
plt.axvline(dec_median_sep, 0, np.max(n), c='k', lw=4, label='Median Separation = %.2f milliarcsec' % (1000*dec_median_sep))
plt.axvline(0.05, 0, np.max(n), label='Requirement =  50 milliarcsec', c='r', lw=4)
plt.title('Test of Absolute Astrometry dec')
plt.xlabel('Distance to match (arcsec)')
plt.ylabel('Number of Gaia Objects in Visit')
plt.legend()

The requirements are satisfied if both RA and dec median values are less than 50 milliarcseconds.

### Test against requirements

If these fail with a new version of the stack our CI testing of notebooks will also fail and alert us.

In [ ]:
class RequirementFailure(ValueError):
    "Requirement not met."

In [ ]:
# Set up for potential error messages
error_msg = ""
error_present = False
error_val = 0

In [ ]:
# Test RA. Convert to milliarcsec by multiplying by 1000.
if ra_median_sep*1000. > 50:
    error_present = True
    error_val += 1
    error_msg += str('Error #%i: \n' % error_val + 
                     'Median RA Astrometry Error greater than 50 milliarcsec. Test Value: %.2f mas \n' % (ra_median_sep*1000.))

In [ ]:
# Test dec. Convert to milliarcsec by multiplying by 1000.
if dec_median_sep*1000. > 50:
    error_present = True
    error_val += 1
    error_msg += str('Error #%i: \n' % error_val +
                     'Median dec Astrometry Error greater than 50 milliarcsec. Test Value: %.2f mas' % (dec_median_sep*1000.))

In [ ]:
if error_present is True:
    error_msg = str('%i Total Errors: \n' % error_val + error_msg)

### Test Failure Diagnostics

Here we run code to help identify where problems are occurring. In this test we look at the results in each CCD to identify if there are issues within a particular subset of CCDs.

In [ ]:
ccd_ids = []
ccd_median_sep_ra = []
ccd_median_sep_dec = []
ccd_num_sources = []
matched_ccd_nums = hsc_sources_df['ccd'].iloc[base_idx]
for ccd_on in np.unique(matched_ccd_nums):
    ccd_ids.append(ccd_on)
    ccd_idx = np.where(matched_ccd_nums == ccd_on)
    ccd_median_sep_ra.append(np.median(np.abs(sep_ra)[ccd_on]))
    ccd_median_sep_dec.append(np.median(np.abs(sep_dec)[ccd_on]))
    ccd_num_sources.append(len(ccd_idx[0]))
ccd_median_sep_ra = np.array(ccd_median_sep_ra)
ccd_median_sep_dec = np.array(ccd_median_sep_dec)
sep_ra_colors = (ccd_median_sep_ra < 50) * 1.
sep_dec_colors = (ccd_median_sep_dec < 50) * 1.

In [ ]:
fig = plt.figure(figsize=(20, 8))

fig.add_subplot(1,2,1)
plt.scatter(ccd_ids, ccd_median_sep_ra, c=sep_ra_colors, vmin=0.1, vmax=0.9, cmap=plt.cm.bwr_r)
plt.xlabel('CCD Number')
plt.ylabel('Median Separation of matched objects in CCD (arcsec)')
plt.title('Median RA Separation by CCD')

fig.add_subplot(1,2,2)
plt.scatter(ccd_ids, ccd_median_sep_dec, c=sep_dec_colors, vmin=0.1, vmax=0.9, cmap=plt.cm.bwr_r)
plt.xlabel('CCD Number')
plt.title('Median dec Separation by CCD')

In [ ]:
if error_present is True:
    failing_idx_ra = np.where(ccd_median_sep_ra > 50.)[0]
    failing_idx_dec = np.where(ccd_median_sep_dec > 50.)[0]

    diag_msg = "\nAdditional Diagnostic Information: \n"
    if len(failing_idx_ra) > 0:
        diag_msg += 'CCD(s) #'
        for fail_ccd in np.array(ccd_ids)[failing_idx_ra]:
            diag_msg += '%s ' % fail_ccd
        diag_msg += 'failing 50 mas requirement in RA.\n'

    if len(failing_idx_dec) > 0:
        diag_msg += 'CCD(s) #'
        for fail_ccd in np.array(ccd_ids)[failing_idx_dec]:
            diag_msg += '%s ' % fail_ccd
        diag_msg += 'failing 50 mas requirement in dec.\n'


### Report Error Messages

Set `fail_in_notebook` to `True` if you want notebook CI to fail based upon test output. Otherwise notebook will run all the way through in CI and produce plots that will be pushed to github and only fail when changes in the DM Stack cause the testing code to break.

In [ ]:
fail_in_notebook = False

In [ ]:
if fail_in_notebook is True:
    if error_present is True:
        raise RequirementFailure(str(error_msg + '\n' + diag_msg))
else:
    if error_present is True:
        print(str(error_msg + '\n' + diag_msg))